# Bin packing

In [1]:
import opvious
import opvious.modeling as om

client = opvious.Client.from_environment()

## Problem formulation

In [2]:
class BinPacking(om.Model):
    """Sample bin-packing formulation"""

    items = om.Dimension()
    weight = om.Parameter.non_negative(items)
    bins = om.interval(1, om.size(items))
    bin_max_weight = om.Parameter.non_negative()

    assigned = om.Variable.indicator(items, bins)
    bin_used = om.Variable.indicator(bins)
    
    @om.objective
    def minimize_bins_used(self):
        return om.total(self.bin_used(b) for b in self.bins)
    
    @om.constraint
    def each_item_is_assigned_once(self):
        for i in self.items:
            yield om.total(self.assigned(i, b) for b in self.bins) == 1
            
    @om.constraint
    def bins_with_assignments_are_used(self):
        for i, b in om.cross(self.items, self.bins):
            yield self.assigned(i, b) <= self.bin_used(b)
            
    @om.constraint
    def bin_weights_are_below_max(self):
        for b in self.bins:
            bin_weight = om.total(self.weight(i) * self.assigned(i, b) for i in self.items)
            yield bin_weight <= self.bin_max_weight()
            
bp = BinPacking()
bp.specification()

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">BinPacking</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^d_\mathrm{items}&: I \\
  \S^p_\mathrm{weight}&: w \in \mathbb{R}_+^{I} \\
  \S^p_\mathrm{binMaxWeight}&: w^\mathrm{binMax} \in \mathbb{R}_+ \\
  \S^v_\mathrm{assigned}&: \alpha \in \{0, 1\}^{I \times \{ 1 \ldots \lvert I \rvert \}} \\
  \S^v_\mathrm{binUsed}&: \psi^\mathrm{bin} \in \{0, 1\}^{\{ 1 \ldots \lvert I \rvert \}} \\
  \S^o_\mathrm{minimizeBinsUsed}&: \min \sum_{x \in \{ 1 \ldots \lvert I \rvert \}} \psi^\mathrm{bin}_{x} \\
  \S^c_\mathrm{eachItemIsAssignedOnce}&: \forall i \in I, \sum_{x \in \{ 1 \ldots \lvert I \rvert \}} \alpha_{i,x} = 1 \\
  \S^c_\mathrm{binsWithAssignmentsAreUsed}&: \forall i \in I, x \in \{ 1 \ldots \lvert I \rvert \}, \alpha_{i,x} \leq \psi^\mathrm{bin}_{x} \\
  \S^c_\mathrm{binWeightsAreBelowMax}&: \forall x \in \{ 1 \ldots \lvert I \rvert \}, \sum_{i \in I} w_{i} \alpha_{i,x} \leq w^\mathrm{binMax} \\
\end{align*}
$$
</div>
</details>
</div>

## Optimal solutions

In [3]:
async def optimal_assignment(bin_max_weight, item_weights):
    """Returns the optimal assignment from item name to bin index"""
    res = await client.run_solve(
        specification=bp.specification(),
        parameters={'weight': item_weights, 'binMaxWeight': bin_max_weight},
        assert_feasible=True,
    )
    df = res.outputs.variable('assigned')
    return dict(df.index)

In [4]:
await optimal_assignment(15, {
    'light': 5,
    'medium': 10,
    'heavy': 15,
})

{'light': 2, 'medium': 2, 'heavy': 1}